In [1]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
# %matplotlib inline

path = "/gpfs/bsc_home/xpliu/inference/vision/classification_and_detection/output-exp10/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.head()

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,0,1,offline,2,10,124.954906,98.902522,98.907836,157.877183,177.378580,187.143268,194.944723,196.619989
1,resnet,none,0,1,offline,4,10,132.407343,93.298632,93.241008,148.973172,167.605823,176.799359,184.275411,185.552759
2,resnet,none,0,1,offline,8,10,144.345208,86.209204,86.158082,137.098597,154.086144,162.588653,169.507216,170.206224
3,resnet,none,0,1,offline,10,10,146.247073,85.456013,85.493713,135.818560,152.484820,160.857246,167.471080,168.002854
4,resnet,none,0,1,offline,16,10,146.507946,86.325441,86.447858,136.151244,152.907493,160.896365,167.298839,167.713889


In [2]:
len(df)

108

In [3]:
df['benchmark'].unique()

array(['resnet'], dtype=object)

In [4]:
df['k8senv'].unique()

array(['none', 'cpumemory', 'cpumem'], dtype=object)

In [5]:
type(df['qps'][0])

numpy.float64

# Graph

In [6]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
%matplotlib inline
dfg = df
dfg

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,0,1,offline,2,10,124.954906,98.902522,98.907836,157.877183,177.378580,187.143268,194.944723,196.619989
1,resnet,none,0,1,offline,4,10,132.407343,93.298632,93.241008,148.973172,167.605823,176.799359,184.275411,185.552759
2,resnet,none,0,1,offline,8,10,144.345208,86.209204,86.158082,137.098597,154.086144,162.588653,169.507216,170.206224
3,resnet,none,0,1,offline,10,10,146.247073,85.456013,85.493713,135.818560,152.484820,160.857246,167.471080,168.002854
4,resnet,none,0,1,offline,16,10,146.507946,86.325441,86.447858,136.151244,152.907493,160.896365,167.298839,167.713889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,resnet,cpumem,0,32,offline,4,10,184.653980,66.758687,66.822818,106.345441,119.661200,126.176245,131.508532,132.634964
104,resnet,cpumem,0,32,offline,8,10,185.623013,66.830171,66.697705,105.985718,118.972311,125.615467,130.709245,131.857241
105,resnet,cpumem,0,32,offline,10,10,185.123461,66.997507,67.241889,106.460580,119.417448,125.834489,131.147465,132.306657
106,resnet,cpumem,0,32,offline,16,10,183.341778,68.224539,67.903520,108.374996,121.460827,127.790155,133.195334,133.816796


In [24]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.pdfgen import canvas

from numpy import percentile, mean, std
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot

import plotly.io as pio
pio.templates.default = "none"
# pio.templates.default = "simple_white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
# import plotly.express as px

for x in ['qps','mean','p99']:
    for cbatch in [2,4,8,10,16,32]:
        for model_name in ['resnet']:
            list_traces = []
            for numctn in [1,2,4,8,16,32]: 
                dfg_bench = dfg[(dfg['benchmark']== model_name)  & (dfg['cbatch']==cbatch) & (dfg['numctn']==numctn) &\
                            ((dfg['k8senv']=='none') |\
                             (dfg['k8senv']=='cpumem')| (dfg['k8senv']=='cpumemory'))]
#                 print(dfg_bench)


                #data
                trace = go.Scatter(
                    name="ctn"+str(numctn),
                    x=[ 'none','cpumem','cpu'],
                    y= dfg_bench[x].values.tolist(),
        #             error_y = dict(type='data', array=df1_bench['model_time_std'].values.tolist()),
                    opacity=0.7,
                    mode='markers'
                )
                list_traces.append(trace)



            #layout
            layout = go.Layout(
                title={
                    'text': "exp9= inter64,intra32 -> inter2,intra1; restapi=32" ,
                    'y':0.85,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                width=1000,
                height=460,
                xaxis=dict(
                    title= "scenarios",
                    tickvals=[ 'none','cpu','cpumem'],
                    titlefont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    )
                ),
                yaxis=dict(
                    title= x+"batch"+str(cbatch),
                    automargin=True,
                    titlefont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),    
                ),
                legend=dict(
                    x=1.0,
                    y=1.0,
                    bgcolor='rgba(255, 255, 255, 0)',
                    bordercolor='rgba(255, 255, 255, 0)',
                    font = dict( size = 28)
                ),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.08
        #         legend_orientation="h"
            )

        data = list_traces
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(legend_orientation="h")
        fig.show()    

    #     fig.write_image(path + model_name + k8senv +'.pdf')

In [23]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.pdfgen import canvas

from numpy import percentile, mean, std
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot

import plotly.io as pio
pio.templates.default = "none"
# pio.templates.default = "simple_white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
# import plotly.express as px

for x in ['qps']:
#     for cbatch in [2,4,8,10,16,32]:
#     for k_env in ["none","cpumem","cpumemory"]:
        for model_name in ['resnet']:
            list_traces = []
            for numctn in [1,2,4,8,16,32]: 
                dfg_bench = dfg[(dfg['benchmark']== model_name)  & (dfg['cbatch']==32/numctn) & (dfg['numctn']==numctn) &\
                            ((dfg['k8senv']=='none') |\
                             (dfg['k8senv']=='cpumem')| (dfg['k8senv']=='cpumemory'))]
                print(dfg_bench)


                #data
                trace = go.Scatter(
                    name="ctn"+str(numctn),
                    x=[ 'none','cpu','cpumem'],
                    y= dfg_bench[x].values.tolist(),
        #             error_y = dict(type='data', array=df1_bench['model_time_std'].values.tolist()),
                    opacity=0.7,
                    mode='markers'
                )
                list_traces.append(trace)



            #layout
            layout = go.Layout(
                title={
                    'text': "exp9= inter64,intra32 -> inter2,intra1; restapi=32(with batch difference)" ,
                    'y':0.85,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                width=1000,
                height=460,
                xaxis=dict(
                    title= "scenarios",
                    tickvals=[ 'none','cpumem','cpu'],
                    titlefont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    )
                ),
                yaxis=dict(
                    title= x,
                    automargin=True,
                    titlefont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),    
                ),
                legend=dict(
                    x=1.0,
                    y=1.0,
                    bgcolor='rgba(255, 255, 255, 0)',
                    bordercolor='rgba(255, 255, 255, 0)',
                    font = dict( size = 28)
                ),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.08
        #         legend_orientation="h"
            )

        data = list_traces
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(legend_orientation="h")
        fig.show()    

    #     fig.write_image(path + model_name + k8senv +'.pdf')

   benchmark     k8senv  sbatch  numctn cscenario  cbatch  reps         qps  \
5     resnet       none       0       1   offline      32    10  146.429732   
41    resnet  cpumemory       0       1   offline      32    10  150.230382   
77    resnet     cpumem       0       1   offline      32    10  150.119391   

         mean        p50         p80         p90         p95         p99  \
5   88.176143  88.750253  137.889168  154.814128  163.550464  167.694973   
41  85.969070  86.918861  135.028602  150.909315  159.345679  163.427776   
77  86.171487  86.631498  135.140977  150.686917  159.502862  163.586576   

          p999  
5   167.790451  
41  163.546571  
77  163.663587  
   benchmark     k8senv  sbatch  numctn cscenario  cbatch  reps         qps  \
10    resnet       none       0       2   offline      16    10  182.813363   
46    resnet  cpumemory       0       2   offline      16    10  188.378037   
82    resnet     cpumem       0       2   offline      16    10  188.0036